In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
pd.options.display.max_rows = 250


import plotly.express as px
import seaborn as sns

In [2]:
files = glob.glob('../mens-march-mania-2022/MDataFiles_Stage2/*')
[print(i,'\m/',f) for i,f in enumerate(files)];

0 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneyDetailedResults.csv
1 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneyCompactResults.csv
2 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MSeasons.csv
3 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MMasseyOrdinals_thruDay128.csv
4 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonDetailedResults.csv
5 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneySlots.csv
6 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MGameCities.csv
7 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MConferenceTourneyGames.csv
8 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/Cities.csv
9 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonCompactResults.csv
10 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneySeedRoundSlots.csv
11 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MTeamConferences.csv
12 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MSampleSubmissionStage2.csv
13 \m/ ../mens-march-mania-20

In [3]:
#1) Use ../mens-march-mania-2022/MDataFiles_Stage2/MTeams.csv to creat team df
teams = pd.read_csv(files[15],usecols=['TeamID','TeamName'])
#teams

# 2) NCAA Basketball Season Data (../mens-march-mania-2022/MDataFiles_Stage2/MSeasons.csv)
seasons = pd.read_csv(files[2])


# Conferences
conferences = pd.read_csv(files[11],usecols=['ConfAbbrev'])

# 3) Seeds in tournament
seeds = pd.read_csv(files[16])
#seeds.tail()

# 4) Merge Teams and Tourney Seed DF's
seeds = pd.merge(seeds, teams,on='TeamID',how='left')
seeds

#seeds.to_csv('GroupProj_Seeds_DF.csv', encoding='utf-8', index=False)

,Season,Seed,TeamID,TeamName
0,1985,W01,1207,Georgetown
1,1985,W02,1210,Georgia Tech
2,1985,W03,1228,Illinois
3,1985,W04,1260,Loyola-Chicago
4,1985,W05,1374,SMU
...,...,...,...,...
2417,2022,Z13,1151,Chattanooga
2418,2022,Z14,1255,Longwood
2419,2022,Z15,1174,Delaware
2420,2022,Z16a,1136,Bryant


In [4]:
# 5) Separating regions and Seeds (Ex: making Seed Z16 go to 16)

seeds['Region'] = seeds['Seed'].apply(lambda x: x[0][:1])
seeds['Seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))

print(seeds.head())
print(seeds.shape)

   Season  Seed  TeamID        TeamName Region
0    1985     1    1207      Georgetown      W
1    1985     2    1210    Georgia Tech      W
2    1985     3    1228        Illinois      W
3    1985     4    1260  Loyola-Chicago      W
4    1985     5    1374             SMU      W
(2422, 5)


In [5]:
# 6) Conferences view - merging Conferences and Teams /// Is not correct// returning duplicates
## May not need

# conferences = pd.read_csv(files[11],usecols=['TeamID','ConfAbbrev'])
# conferences_view = pd.merge(teams,conferences,on='TeamID',how='left')
# conferences_view = conferences_view.reset_index().drop_duplicates()

# #columns = ['TeamID', 'TeamName', 'ConfAbbrev']
# #conferences_view.columns
# conferences_view

In [6]:
# 7)
# Dropping NumOT -  (Overtime data can remove)
## Compliling RegularSeasonResultsGamelogs and Tourney Game data -
### Using pd.concat to arrange/merge dataframes -
####Add a column called Score Differential = WScore - LScore

SeasonResults = pd.read_csv(files[9])
SeasonResults = SeasonResults[SeasonResults['Season'] >= 2005].reset_index(drop=True) 
#SeasonResults = SeasonResults.copy()

columns = [['DayNum','GameID', 'WScore', 'WTeamID','LScore','LTeamID','WLoc', 'result','WL','Score_Diff']]

## 7)
##Create a unique GameID for later
SeasonResults['GameID'] = np.arange(1,len(SeasonResults)+1)

SeasonResultsWin = SeasonResults[['Season','DayNum','GameID','WTeamID','WScore','LTeamID','LScore','WLoc']].copy()

SeasonResultsWin.columns = ['Season','DayNum','GameID','WTeamID','WScore','LTeamID','LScore','WLoc']
SeasonResultsWin['result'] = 1

SeasonResultsLose = SeasonResults[['Season','DayNum','GameID','WTeamID','WScore','LTeamID','LScore','WLoc']].copy()
SeasonResultsLose.columns = ['Season','DayNum','GameID','LTeamID','LScore','WTeamID','WScore','WLoc']
SeasonResultsLose['result'] = 0

SeasonResultsTeams = pd.concat([SeasonResultsWin,SeasonResultsLose],axis=0,sort=True).sort_values('GameID')

# Add a (Score_Diff) score differential to create a metric that will tell us strength by points won by

SeasonResultsTeams['WL'] = np.where(SeasonResultsTeams['WScore']-SeasonResultsTeams['LScore']>0,'W','L')

SeasonResultsTeams['Score_Diff'] = SeasonResultsTeams['WScore'] - SeasonResultsTeams['LScore']

#SeasonResultsTeams['season_g_id'] = SeasonResultsTeams.Season.astype(str) + SeasonResultsTeams.TeamID.astype(str)

#Reorder columns
#SeasonResultsTeams.columns = columns
#columns = [['DayNum','GameID', 'Score', 'TeamID','Score','LTeamID','LScore','WLoc', 'result','WL','Score_Diff']]

#SeasonResultsTeams.to_csv('GroupProj_RegSeasII_DF.csv', encoding='utf-8', index=False)

SeasonResultsTeams = SeasonResultsTeams[SeasonResultsTeams['Season'] >= 2005].reset_index(drop=True)           
SeasonResultsTeams = SeasonResultsTeams.copy()
## 8)
### DDrop NUMOT and Win Location {WLoc}
SeasonResultsTeams.drop(['WLoc'], axis=1, inplace=True)
SeasonResultsTeams 

#185690 rows × 11 columns

,DayNum,GameID,LScore,LTeamID,Season,WScore,WTeamID,result,WL,Score_Diff
0,10,1,69,1105,2005,79,1128,1,W,10
1,10,1,79,1128,2005,69,1105,0,L,-10
2,10,2,78,1143,2005,61,1415,0,L,-17
3,10,2,61,1415,2005,78,1143,1,W,17
4,10,3,51,1236,2005,69,1206,1,W,18
...,...,...,...,...,...,...,...,...,...,...
185685,132,92843,62,1172,2022,64,1350,1,W,2
185686,132,92844,65,1397,2022,50,1401,0,L,-15
185687,132,92844,50,1401,2022,65,1397,1,W,15
185688,132,92845,64,1343,2022,66,1463,1,W,2


In [7]:
games_won = SeasonResults.groupby(['Season', 'WTeamID']).count()
games_won = games_won.reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns={"DayNum": "Wins", "WTeamID": "TeamID"})

games_won.tail()

# 6231 rows × 3 columns

,Season,TeamID,Wins
6226,2022,1468,16
6227,2022,1469,9
6228,2022,1470,11
6229,2022,1471,11
6230,2022,1472,7


In [8]:
games_lost = SeasonResults.groupby(['Season', 'LTeamID']).count()
games_lost = games_lost.reset_index()[['Season', 'LTeamID', 'DayNum']].rename(columns={"DayNum": "Losses", "LTeamID": "TeamID"})

games_lost.head()

,Season,TeamID,Losses
0,2005,1102,12
1,2005,1103,10
2,2005,1104,7
3,2005,1105,13
4,2005,1106,15


In [9]:
## Compile Score_Differential to win_differential and loss_differential


win_differential = SeasonResultsTeams.groupby(['Season', 'WTeamID']).mean().reset_index()
win_differential = win_differential[['Season', 'WTeamID', 'Score_Diff']].rename(columns={"Score_Diff": "Win_Differential", "WTeamID": "TeamID"})


loss_differential = SeasonResultsTeams.groupby(['Season', 'LTeamID']).mean().reset_index()
loss_differential = loss_differential[['Season', 'LTeamID', 'Score_Diff']].rename(columns={"Score_Diff": "Loss_Differential", "LTeamID": "TeamID"})


#win_differential
#loss_differential

In [10]:
#Merge winning and losing season results 

#split features by 


winning_season_features = SeasonResults.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
losing_season_features = SeasonResults.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})

season_results_features = pd.concat([winning_season_features, losing_season_features], 0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)



season_results_features = season_results_features.merge(games_won, on=['Season', 'TeamID'], how='left')
season_results_features = season_results_features.merge(games_lost, on=['Season', 'TeamID'], how='left')
season_results_features = season_results_features.merge(win_differential, on=['Season', 'TeamID'], how='left')
season_results_features = season_results_features.merge(loss_differential, on=['Season', 'TeamID'], how='left')
#season_results_features = season_results_features.merge(massey_ratings, on=['Season', 'TeamID'], how='left')

#season_results_features.fillna(0, inplace=True)  

#season_results_features.astype(int)

season_results_features


/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  if __name__ == '__main__':


,Season,TeamID,Wins,Losses,Win_Differential,Loss_Differential
0,2005,1102,17.0,12.0,6.103448,-6.103448
1,2005,1103,18.0,10.0,5.357143,-5.357143
2,2005,1104,23.0,7.0,10.600000,-10.600000
3,2005,1105,16.0,13.0,1.275862,-1.275862
4,2005,1106,15.0,15.0,-3.333333,3.333333
...,...,...,...,...,...,...
6234,2022,1468,16.0,13.0,-2.482759,2.482759
6235,2022,1469,9.0,18.0,-8.000000,8.000000
6236,2022,1470,11.0,17.0,-2.678571,2.678571
6237,2022,1471,11.0,16.0,-3.259259,3.259259


In [11]:
season_results_features['WinRatio'] = season_results_features['Wins'] / (season_results_features['Wins'] + season_results_features['Losses'])
season_results_features['Score_Diff_Avg'] = (
    (season_results_features['Wins'] * season_results_features['Win_Differential'] - 
    season_results_features['Losses'] * season_results_features['Loss_Differential'])
    / (season_results_features['Wins'] + season_results_features['Losses'])
)

season_results_features.drop(['Win_Differential', 'Loss_Differential'], axis=1, inplace=True)


season_results_features

,Season,TeamID,Wins,Losses,WinRatio,Score_Diff_Avg
0,2005,1102,17.0,12.0,0.586207,6.103448
1,2005,1103,18.0,10.0,0.642857,5.357143
2,2005,1104,23.0,7.0,0.766667,10.600000
3,2005,1105,16.0,13.0,0.551724,1.275862
4,2005,1106,15.0,15.0,0.500000,-3.333333
...,...,...,...,...,...,...
6234,2022,1468,16.0,13.0,0.551724,-2.482759
6235,2022,1469,9.0,18.0,0.333333,-8.000000
6236,2022,1470,11.0,17.0,0.392857,-2.678571
6237,2022,1471,11.0,16.0,0.407407,-3.259259


# Tourney results
This file identifies the game-by-game NCAA® tournament results for all seasons of historical data. The data is formatted exactly like the MRegularSeasonCompactResults data. All games will show up as neutral site (so WLoc is always N). Note that this tournament game data also includes the play-in games (which always occurred on day 134/135) for those years that had play-in games. Thus each season you will see between 63 and 67 games listed, depending on how many play-in games there were.

In [12]:
TourneyResults = pd.read_csv(files[1])
TourneyResults.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

TourneyResults.tail()


,Season,DayNum,WTeamID,WScore,LTeamID,LScore
2312,2021,148,1211,85,1425,66
2313,2021,148,1417,51,1276,49
2314,2021,152,1124,78,1222,59
2315,2021,152,1211,93,1417,90
2316,2021,154,1124,86,1211,70


In [13]:
# Need to merge massey rankings into main df

massey_ratings = pd.read_csv(files[3])
massey_ratings.drop('SystemName', axis=1, inplace=True)


#massey_ratings = SeasonResults.groupby(['Season', 'TeamID']).reset_index()
#massey_ratings = massey_ratings[['Season', 'RankingDayNum', 'TeamID']].rename(columns={"OrdinalRank": "Ordinal Rank"})
massey_ratings.head()

,Season,RankingDayNum,TeamID,OrdinalRank
0,2003,35,1102,159
1,2003,35,1103,229
2,2003,35,1104,12
3,2003,35,1105,314
4,2003,35,1106,260


In [14]:
df = TourneyResults.copy()
df = df[df['Season'] >= 2005].reset_index(drop=True)

#df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,TeamName_x,Region_x,SeedL,TeamName_y,Region_y
0,2005,134,1324,79,1105,69,16,Oakland,Z,16,Alabama A&M,Z
1,2005,136,1112,66,1429,53,3,Arizona,X,14,Utah St,X
2,2005,136,1130,85,1335,65,4,Boston College,X,13,Penn,X
3,2005,136,1153,76,1234,64,7,Cincinnati,Y,10,Iowa,Y
4,2005,136,1211,74,1457,64,3,Gonzaga,W,14,Winthrop,W
...,...,...,...,...,...,...,...,...,...,...,...,...
1048,2021,148,1211,85,1425,66,1,Gonzaga,X,6,USC,X
1049,2021,148,1417,51,1276,49,11,UCLA,W,1,Michigan,W
1050,2021,152,1124,78,1222,59,1,Baylor,Z,2,Houston,Y
1051,2021,152,1211,93,1417,90,1,Gonzaga,X,11,UCLA,W


In [29]:
df = pd.merge(
    df, 
    seeds, 
    how='left', 
    left_on=['Season', 'WTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedW'})


df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,TeamName_x,Region_x,SeedL,...,Losses_L,WinRatioW,Score_Difference_Average,Wins_W,Losses_L,WinRatioW,Score_Difference_Average,SeedW,TeamName,Region
0,2005,134,1324,79,1105,69,16,Oakland,Z,16,...,18.0,0.379310,-3.137931,11.0,18.0,0.379310,-3.137931,16,Oakland,Z
1,2005,136,1112,66,1429,53,3,Arizona,X,14,...,6.0,0.818182,10.121212,27.0,6.0,0.818182,10.121212,3,Arizona,X
2,2005,136,1130,85,1335,65,4,Boston College,X,13,...,4.0,0.857143,8.107143,24.0,4.0,0.857143,8.107143,4,Boston College,X
3,2005,136,1153,76,1234,64,7,Cincinnati,Y,10,...,7.0,0.774194,12.000000,24.0,7.0,0.774194,12.000000,7,Cincinnati,Y
4,2005,136,1211,74,1457,64,3,Gonzaga,W,14,...,4.0,0.862069,9.896552,25.0,4.0,0.862069,9.896552,3,Gonzaga,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,2021,148,1211,85,1425,66,1,Gonzaga,X,6,...,NaN,NaN,NaN,26.0,NaN,NaN,NaN,1,Gonzaga,X
1049,2021,148,1417,51,1276,49,11,UCLA,W,1,...,9.0,0.653846,4.346154,17.0,9.0,0.653846,4.346154,11,UCLA,W
1050,2021,152,1124,78,1222,59,1,Baylor,Z,2,...,2.0,0.916667,17.958333,22.0,2.0,0.916667,17.958333,1,Baylor,Z
1051,2021,152,1211,93,1417,90,1,Gonzaga,X,11,...,NaN,NaN,NaN,26.0,NaN,NaN,NaN,1,Gonzaga,X


In [30]:
df = pd.merge(
    df, 
    seeds, 
    how='left', 
    left_on=['Season', 'LTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedL'})



df

/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Region_x', 'TeamName_x'} in the result is deprecated and will raise a MergeError in a future version.
  


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,TeamName_x,Region_x,SeedL,...,Wins_W,Losses_L,WinRatioW,Score_Difference_Average,SeedW,TeamName_x,Region_x,SeedL,TeamName_y,Region_y
0,2005,134,1324,79,1105,69,16,Oakland,Z,16,...,11.0,18.0,0.379310,-3.137931,16,Oakland,Z,16,Alabama A&M,Z
1,2005,136,1112,66,1429,53,3,Arizona,X,14,...,27.0,6.0,0.818182,10.121212,3,Arizona,X,14,Utah St,X
2,2005,136,1130,85,1335,65,4,Boston College,X,13,...,24.0,4.0,0.857143,8.107143,4,Boston College,X,13,Penn,X
3,2005,136,1153,76,1234,64,7,Cincinnati,Y,10,...,24.0,7.0,0.774194,12.000000,7,Cincinnati,Y,10,Iowa,Y
4,2005,136,1211,74,1457,64,3,Gonzaga,W,14,...,25.0,4.0,0.862069,9.896552,3,Gonzaga,W,14,Winthrop,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,2021,148,1211,85,1425,66,1,Gonzaga,X,6,...,26.0,NaN,NaN,NaN,1,Gonzaga,X,6,USC,X
1049,2021,148,1417,51,1276,49,11,UCLA,W,1,...,17.0,9.0,0.653846,4.346154,11,UCLA,W,1,Michigan,W
1050,2021,152,1124,78,1222,59,1,Baylor,Z,2,...,22.0,2.0,0.916667,17.958333,1,Baylor,Z,2,Houston,Y
1051,2021,152,1211,93,1417,90,1,Gonzaga,X,11,...,26.0,NaN,NaN,NaN,1,Gonzaga,X,11,UCLA,W


In [37]:
# # Optional create a function to weight higher seeds

# def treat_seed(seed):
#     return int(re.sub("[^0-9]", "", seed))


# df['SeedW'] = df['SeedW'].apply(treat_seed)
# df['SeedL'] = df['SeedL'].apply(treat_seed)

In [35]:
df = pd.merge(
    df,
    season_results_features,
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'Wins': 'Wins_W',
    'Losses': 'Losses_L',
    'Win_Differential': 'Win_Diff',
    'Loss_Differential': 'Loss_Diff',
    'WinRatio': 'WinRatioW',
    'Score_Diff_Avg': 'Score_Difference_Average',
}).drop(columns='TeamID', axis=1).drop_duplicates()

df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,TeamName_x,Region_x,SeedL,...,TeamName_y,Region_y,Wins_W,Losses_L,WinRatioW,Score_Difference_Average,Wins_W,Losses_L,WinRatioW,Score_Difference_Average
0,2005,134,1324,79,1105,69,16,Oakland,Z,16,...,Alabama A&M,Z,11.0,18.0,0.379310,-3.137931,11.0,18.0,0.379310,-3.137931
1,2005,136,1112,66,1429,53,3,Arizona,X,14,...,Utah St,X,27.0,6.0,0.818182,10.121212,27.0,6.0,0.818182,10.121212
2,2005,136,1130,85,1335,65,4,Boston College,X,13,...,Penn,X,24.0,4.0,0.857143,8.107143,24.0,4.0,0.857143,8.107143
3,2005,136,1153,76,1234,64,7,Cincinnati,Y,10,...,Iowa,Y,24.0,7.0,0.774194,12.000000,24.0,7.0,0.774194,12.000000
4,2005,136,1211,74,1457,64,3,Gonzaga,W,14,...,Winthrop,W,25.0,4.0,0.862069,9.896552,25.0,4.0,0.862069,9.896552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,2021,148,1211,85,1425,66,1,Gonzaga,X,6,...,USC,X,26.0,NaN,NaN,NaN,26.0,NaN,NaN,NaN
1049,2021,148,1417,51,1276,49,11,UCLA,W,1,...,Michigan,W,17.0,9.0,0.653846,4.346154,17.0,9.0,0.653846,4.346154
1050,2021,152,1124,78,1222,59,1,Baylor,Z,2,...,Houston,Y,22.0,2.0,0.916667,17.958333,22.0,2.0,0.916667,17.958333
1051,2021,152,1211,93,1417,90,1,Gonzaga,X,11,...,UCLA,W,26.0,NaN,NaN,NaN,26.0,NaN,NaN,NaN


In [36]:
# 10) Games played each year
games_played = TourneyResults.groupby('Season')['DayNum'].count().to_frame().merge(SeasonResults.groupby('Season')['DayNum'].count().to_frame(),on='Season')
games_played.rename(columns={"DayNum_x": "Tournament Games", "DayNum_y": "Regular Season Games"})


games_played

,DayNum_x,DayNum_y
Season,,
2005,64,4675
2006,64,4757
2007,64,5043
2008,64,5163
2009,64,5249
2010,64,5263
2011,67,5246
2012,67,5253
2013,67,5320
